In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark.sql('select current_date() as spark_started ').show()

+-------------+
|spark_started|
+-------------+
|   2022-10-29|
+-------------+



In [23]:
spark.stop()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover
# from pyspark.ml.linalg import DenseVector, SparseVector
# from pyspark.sql.types import FloatType, StructType
import numpy as np

# Чтение файлов из hdfs и формирование 'target'

In [5]:
target = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [6]:
# задача взять эти Ид и по ним найти рекомендации
my_en = [23126, 21617]
my_es = [16627, 11556]
my_ru = [16704, 13702]
my = my_en + my_es + my_ru
my

[23126, 21617, 16627, 11556, 16704, 13702]

In [7]:
# возьмем только 3 колонки из Задания и положим в Табу из 3х колонок
rdd = spark.sparkContext.parallelize(target)
df= spark.createDataFrame(rdd).toDF(*['id', 'lang', 'name'])

In [8]:
courses = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [10]:
courses.columns

['cat', 'desc', 'id', 'lang', 'name', 'provider']

In [11]:
target = df.join(courses, df.columns, 'inner')

# TF-IDF

In [12]:
# получим мусорные слова
stop_word = StopWordsRemover.loadDefaultStopWords ("english") \
+  StopWordsRemover.loadDefaultStopWords ("russian") \
+ StopWordsRemover.loadDefaultStopWords ("spanish")

In [13]:

def my_recommend(df, target, stop):
    
# токенайзер особый
    regexTokenizer = RegexTokenizer(inputCol="desc", outputCol="words", pattern=r"[1234567890?!.)(,\s]")
    regexTokenized = regexTokenizer.transform(df)
    
    
# убираем слова
    remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=stop)
    remover = remover.transform(regexTokenized)
    
# преобразуем в вектор ТФ (частот)
    hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
    featurizedData = hashingTF.transform(remover)
    
# идф - вектор "не встречаемости" в других курсах
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    
# трансформируем TF-IDF
    rescaledData = idfModel.transform(featurizedData)
    
    resault = rescaledData
    return resault

In [15]:
result__tf_idf = my_recommend(courses, target, stop_word)

In [16]:
result__tf_idf.columns

['cat',
 'desc',
 'id',
 'lang',
 'name',
 'provider',
 'words',
 'filtered',
 'rawFeatures',
 'features']

In [17]:
result__tf_idf.show(1, truncate=False)

+-----------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.window import Window

In [19]:
# делаем мой косинусный размер
@f.udf (FloatType ())
def cos_udf (v1, v2):
    return float (v1.dot(v2) / (v1.norm(2) * v2.norm(2)))

In [21]:
# возьмем наши курсы

result_my = result__tf_idf.where("id in (23126, 16627, 21617, 16627, 11556, 16704, 13702)").\
select(col("id").alias("id_my"), col("features").alias("features_my"), col("lang"))

In [22]:
result_1 = result_tfidf.join(result_my, ["lang"], "inner").where ("id <> id_my")

In [23]:
# считаем расстояния
result_2 = result_1.select("lang", "id", "id_my", "features", "features_my", \
                         cos_udf ("features", "features_my").\
                         alias("cos")).where("cos <> 'NaN'").\
orderBy ("lang","id_my", col("cos").desc())

In [24]:
result_2.show(1)

+----+-----+-----+--------------------+--------------------+----------+
|lang|   id|id_my|            features|         features_my|       cos|
+----+-----+-----+--------------------+--------------------+----------+
|  en|21609|21617|(10000,[161,360,5...|(10000,[161,360,5...|0.99598575|
+----+-----+-----+--------------------+--------------------+----------+
only showing top 1 row



In [25]:
# ранжируем, чтобы 10 ближайших расстояний кос*а у 2х векторов было максимальным
windowSpec = Window.partitionBy("id_my").orderBy(col("cos").desc())

final = result_2.select ("lang", "id", "id_my", "features", "features_my", "cos",\
                        f.row_number().over(windowSpec).alias("row_num")).where("row_num <= 10").cache()

In [27]:
final.show(2)

+----+-----+-----+--------------------+--------------------+---------+-------+
|lang|   id|id_my|            features|         features_my|      cos|row_num|
+----+-----+-----+--------------------+--------------------+---------+-------+
|  en|13665|23126|(10000,[51,93,211...|(10000,[87,246,25...|0.5949343|      1|
|  en|14760|23126|(10000,[957,1470,...|(10000,[87,246,25...|0.5889534|      2|
+----+-----+-----+--------------------+--------------------+---------+-------+
only showing top 2 rows



In [28]:
# сгруппируем
final_1 = final.groupBy("id_my").agg(f.collect_list("id")).collect()

In [29]:
final_1

[Row(id_my=23126, collect_list(id)=[13665, 14760, 13782, 20638, 24419, 2724, 15909, 25782, 19270, 13348]),
 Row(id_my=16627, collect_list(id)=[11431, 5687, 17964, 12660, 12247, 17961, 16694, 13551, 5558, 13550]),
 Row(id_my=13702, collect_list(id)=[864, 28074, 8313, 1111, 21079, 8300, 1041, 1110, 13057, 21025]),
 Row(id_my=16704, collect_list(id)=[1247, 1236, 1365, 8186, 1164, 1273, 20288, 1233, 8203, 18331]),
 Row(id_my=11556, collect_list(id)=[16488, 23357, 10447, 468, 19330, 22710, 13461, 21707, 10384, 17910]),
 Row(id_my=21617, collect_list(id)=[21609, 21616, 21608, 21492, 21508, 21624, 21854, 21676, 21623, 21700])]

In [31]:
# сделаю структуру джсона
final_export = {str(x[0]): x[1] for x in final_1}


In [33]:
final_export

{'23126': [13665,
  14760,
  13782,
  20638,
  24419,
  2724,
  15909,
  25782,
  19270,
  13348],
 '16627': [11431, 5687, 17964, 12660, 12247, 17961, 16694, 13551, 5558, 13550],
 '13702': [864, 28074, 8313, 1111, 21079, 8300, 1041, 1110, 13057, 21025],
 '16704': [1247, 1236, 1365, 8186, 1164, 1273, 20288, 1233, 8203, 18331],
 '11556': [16488, 23357, 10447, 468, 19330, 22710, 13461, 21707, 10384, 17910],
 '21617': [21609,
  21616,
  21608,
  21492,
  21508,
  21624,
  21854,
  21676,
  21623,
  21700]}

In [35]:
import json


with open('lab02.json', 'w') as f:
    json.dump(final_export, f, indent=2)
    print("creating file is completed")

creating file is completed


In [36]:
spark.stop()
print('spark stoped')

spark stoped
